# Finding Nested Introns

In [1]:
from collections import defaultdict
from pathlib import Path

import pandas as pd
from pybedtools import BedTool, Interval
from tqdm import tqdm


We define functions for loading different file types with SJ / intron data:

In [2]:
SJ_FILE_COLUMN_NAMES = ['chromosome', 'start', 'end', 'strand', 'intron_motif',
                        'annotated', 'reads_unique', 'reads_multimapped', 'max_overhang']
BED_FILE_COLUMN_NAMES = ['chromosome', 'start', 'end', 'name', 'score', 'strand']


def load_sj_dataframe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path,
                     delimiter='\t',
                     names=SJ_FILE_COLUMN_NAMES)
    # STAR SJ files are 1-based and interval end is inclusive, while .bed files are 0-based and interval end is exclusive.
    # We convert SJ files to .bed-style indexing (note that interval end is unchanged as 0-/1- indexing cancels out with exclusiveness / inclusiveness)
    df['start'] -= 1

    # We omit SJ with unspecified strand    
    df = df[df['strand'] != 0]
    df['strand'] = df['strand'].apply(lambda x: '+' if x == 1 else '-')

    # We keep track how many samples support this SJ # this will be useful when aggregating dataframes from multiple samples
    df['present_in_samples'] = 1
    return df


def load_bed_file_as_dataframe(file_path: Path) -> pd.DataFrame:
    return pd.read_csv(file_path, delimiter='\t', names=BED_FILE_COLUMN_NAMES)

We load SJ files, filter them by some quality control, and aggregate them to a single dataframe:

In [3]:
sj_folder_path = Path('./splice_junction_files')
sj_dataframes = [load_sj_dataframe(path) for path in tqdm(sj_folder_path.iterdir(), desc='Loading SJ files')]


def quality_control_sj(df: pd.DataFrame, min_max_overhang=20, min_unique_reads=5) -> pd.DataFrame:
    df = df[df['max_overhang'] >= min_max_overhang]
    df = df[df['reads_unique'] >= min_unique_reads]
    return df


sj_dataframes = [quality_control_sj(sj) for sj in sj_dataframes]

sj_aggregated = pd.concat(sj_dataframes).groupby(['chromosome', 'start', 'end',
                                                  'strand', 'intron_motif', 'annotated']).agg(
    {'reads_unique': 'sum',
     'reads_multimapped': 'sum',
     'max_overhang': 'max',
     'present_in_samples': 'sum'}).reset_index()

sj_aggregated['present_in_samples_fraction'] = sj_aggregated['present_in_samples'] / len(sj_dataframes)
sj_aggregated

Loading SJ files: 16it [00:01,  8.67it/s]


,chromosome,start,end,strand,intron_motif,annotated,reads_unique,reads_multimapped,max_overhang,present_in_samples,present_in_samples_fraction
0,GL456211.1,54472,55342,+,1,0,5,0,41,1,0.0625
1,GL456211.1,55514,61021,+,1,0,7,0,29,1,0.0625
2,GL456211.1,167988,175871,+,1,1,89,10,50,9,0.5625
3,GL456211.1,176034,176916,+,1,1,37,52,50,5,0.3125
4,GL456211.1,177088,182218,+,1,1,5,1,40,1,0.0625
...,...,...,...,...,...,...,...,...,...,...,...
156901,chrY,90760614,90762078,+,1,1,15,2,46,1,0.0625
156902,chrY,90785979,90793295,+,1,1,273,1479,50,15,0.9375
156903,chrY,90793417,90816348,+,1,1,884,36,50,15,0.9375
156904,chrY,90793680,90816348,+,1,1,187,15,50,15,0.9375


We may further e.g. filter only SJ present in all samples:

In [4]:
sj_aggregated = sj_aggregated[sj_aggregated['present_in_samples_fraction'] == 1]
sj_aggregated

,chromosome,start,end,strand,intron_motif,annotated,reads_unique,reads_multimapped,max_overhang,present_in_samples,present_in_samples_fraction
13,GL456216.1,16407,30698,+,1,1,961,0,50,16,1.0
16,GL456216.1,30806,31058,+,1,1,1138,1,50,16,1.0
17,GL456216.1,31127,31369,+,1,1,661,0,50,16,1.0
19,GL456216.1,31471,34137,+,1,1,1280,0,50,16,1.0
20,GL456216.1,34345,34623,+,1,1,2314,0,50,16,1.0
...,...,...,...,...,...,...,...,...,...,...,...
156882,chrX,169313022,169313587,-,2,1,837,0,50,16,1.0
156883,chrX,169313674,169315601,-,2,1,520,0,50,16,1.0
156886,chrX,169317147,169319437,-,2,1,606,1,50,16,1.0
156887,chrX,169319590,169319975,-,2,1,295,0,50,16,1.0


We define function for finding nested introns and apply it to dataframe with SJ:

In [5]:
def find_nested_introns(df_1: pd.DataFrame, df_2: pd.DataFrame) -> pd.DataFrame:
    """
    Find nested introns (SJ). The potential 'outside' and 'nested' introns are specified by different arguments (
      df_1 resp. df_2), which may be useful if we wish to apply different pre-filtering logic on them.
    :param df_1: Dataframe with potential 'outside' introns.
    :param df_2: Dataframe with potential 'nested' introns.
    :return: Dataframe same as df_1, with added columns 'has_nested_intron' and 'nested_intron_indices'. The column
    'has_nested_intron' contain boolean flag specifying whether this intron contains any nested introns. The column
    'nested_intron_indices' contain indices (from df_2) of nested introns.
    """
    df_output = df_1.copy()

    bedtool_1 = BedTool([Interval(chrom=row['chromosome'],
                                  start=row['start'],
                                  end=row['end'],
                                  name=index,
                                  strand=row['strand']) for index, row in df_1.iterrows()]).sort()

    bedtool_2 = BedTool([Interval(chrom=row['chromosome'],
                                  start=row['start'],
                                  end=row['end'],
                                  name=index,
                                  strand=row['strand']) for index, row in df_2.iterrows()]).sort()

    intersection_bedtool = bedtool_1.intersect(bedtool_2, sorted=True, s=True, wa=True, wb=True)
    intersections_by_query_id = defaultdict(list)
    for intersection_interval in intersection_bedtool:
        intersections_by_query_id[int(intersection_interval.fields[3])].append(int(intersection_interval.fields[9]))

    has_nested_intron: list[bool] = []
    nested_intron_indices: list[list[int]] = []

    for index, row in tqdm(df_1.iterrows(), desc='Finding nested introns', total=len(df_output)):
        intersections_indices = intersections_by_query_id.get(index)
        if not intersections_indices:
            has_nested_intron.append(False)
            nested_intron_indices.append([])
            continue

        intersections_df = df_2.loc[intersections_indices]
        nested_introns_df = intersections_df[intersections_df['start'] > row['start']]
        nested_introns_df = nested_introns_df[nested_introns_df['end'] < row['end']]

        nested_introns_indices = list(nested_introns_df.index)
        has_nested_intron.append(bool(nested_introns_indices))
        nested_intron_indices.append(nested_introns_indices)

    df_output['has_nested_intron'] = has_nested_intron
    df_output['nested_intron_indices'] = nested_intron_indices
    return df_output

In [6]:
sj_aggregated = find_nested_introns(df_1=sj_aggregated, df_2=sj_aggregated)

***** WARNING: File /tmp/pybedtools.gf0xh1r0.tmp has inconsistent naming convention for record:
chr1	4774516	4777524	65	.	-

***** WARNING: File /tmp/pybedtools.gf0xh1r0.tmp has inconsistent naming convention for record:
chr1	4774516	4777524	65	.	-

Finding nested introns: 100%|█████████████████████████████████████| 81774/81774 [02:01<00:00, 674.43it/s]


Some SJ have nested SJ inside them:

In [7]:
sj_with_nested_introns = sj_aggregated[sj_aggregated['has_nested_intron']]
sj_with_nested_introns

,chromosome,start,end,strand,intron_motif,annotated,reads_unique,reads_multimapped,max_overhang,present_in_samples,present_in_samples_fraction,has_nested_intron,nested_intron_indices
2623,chr1,63170914,63176585,-,2,1,1608,1,50,16,1.0,True,[2625]
3730,chr1,82735507,82750519,+,1,1,1005,0,50,16,1.0,True,[3732]
4214,chr1,88071678,88215043,+,1,1,2639,1,50,16,1.0,True,[4219]
4256,chr1,88268026,88275041,-,2,1,956,0,50,16,1.0,True,"[4257, 4258]"
7260,chr1,156041002,156059410,+,1,1,349,2,50,16,1.0,True,[7261]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149076,chr9,83925359,83988775,+,1,0,501,0,50,16,1.0,True,[149082]
150003,chr9,103217518,103219801,-,2,1,474,0,50,16,1.0,True,[150004]
150633,chr9,107659244,107668852,-,2,1,1227,0,50,16,1.0,True,"[150637, 150642]"
151026,chr9,108517240,108528588,+,1,1,469,0,50,16,1.0,True,[151030]


The column 'nested_intron_indices' keeps indices (of second dataframe passed to find_nested_introns() function) which we may use to retrieve the nested SJ:

In [8]:
example_sj = sj_with_nested_introns.iloc[3]
example_nested_introns = sj_aggregated.loc[example_sj['nested_intron_indices']]
print(f"Example SJ: \n{example_sj}")
print(50 * '-')
print("Example nested introns of the SJ above:")
example_nested_introns

Example SJ: 
chromosome                             chr1
start                              88268026
end                                88275041
strand                                    -
intron_motif                              2
annotated                                 1
reads_unique                            956
reads_multimapped                         0
max_overhang                             50
present_in_samples                       16
present_in_samples_fraction             1.0
has_nested_intron                      True
nested_intron_indices          [4257, 4258]
Name: 4256, dtype: object
--------------------------------------------------
Example nested introns of the SJ above:


,chromosome,start,end,strand,intron_motif,annotated,reads_unique,reads_multimapped,max_overhang,present_in_samples,present_in_samples_fraction,has_nested_intron,nested_intron_indices
4257,chr1,88269413,88270201,-,2,1,1315,1,50,16,1.0,False,[]
4258,chr1,88270299,88272728,-,2,1,800,2,43,16,1.0,False,[]


We may find nested SJ inside different reference introns, if we wish to apply different filtering logic on the nested and 'outside' introns.
We may e.g. find SJ that are nested inside introns from the reference genome:

In [10]:
reference_introns = load_bed_file_as_dataframe('./reference_genome/introns.bed')
reference_introns = find_nested_introns(reference_introns, sj_aggregated)

***** WARNING: File /tmp/pybedtools.4cai5zxo.tmp has inconsistent naming convention for record:
chr1	3207317	3213438	192	.	-

***** WARNING: File /tmp/pybedtools.4cai5zxo.tmp has inconsistent naming convention for record:
chr1	3207317	3213438	192	.	-

Finding nested introns: 100%|██████████████████████████████████| 225163/225163 [01:50<00:00, 2039.14it/s]


In [11]:
reference_introns_with_nested_sj = reference_introns[reference_introns['has_nested_intron']]
reference_introns_with_nested_sj

,chromosome,start,end,name,score,strand,has_nested_intron,nested_intron_indices
15838,chr10,5069063,5336948,ENSMUSG00000096054.2,.,-,True,"[10046, 10047, 10048, 10049, 10050, 10051, 100..."
48034,chr12,104731987,104751747,ENSMUSG00000041415.9,.,-,True,[34917]
50423,chr13,17672602,17694638,ENSMUSG00000055137.6,.,-,True,[36699]
76176,chr16,23180009,23218762,ENSMUSG00000068463.3,.,+,True,"[55715, 55717, 55720, 55724]"
79182,chr16,85030365,85056323,ENSMUSG00000022892.10,.,-,True,[57891]
91432,chr18,32135951,32139509,ENSMUSG00000024386.8,.,-,True,"[67593, 67595]"
163394,chr6,40686610,40714600,ENSMUSG00000068587.9,.,+,True,"[118413, 118414, 118415, 118416, 118417, 11841..."
163395,chr6,40714729,40728951,ENSMUSG00000068587.9,.,+,True,"[118437, 118439, 118440, 118442, 118443, 118444]"
175880,chr7,26173604,26186580,ENSMUSG00000040660.6,.,+,True,[126381]


In [12]:
example_intron = reference_introns_with_nested_sj.iloc[3]
example_nested_sj = sj_aggregated.loc[example_intron['nested_intron_indices']]
print(f"Example intron with nested SJ: \n{example_intron}")
print(50 * '-')
print("Nested SJ of the intron above:")
example_nested_sj

Example intron with nested SJ: 
chromosome                                      chr16
start                                        23180009
end                                          23218762
name                             ENSMUSG00000068463.3
score                                               .
strand                                              +
has_nested_intron                                True
nested_intron_indices    [55715, 55717, 55720, 55724]
Name: 76176, dtype: object
--------------------------------------------------
Nested SJ of the intron above:


,chromosome,start,end,strand,intron_motif,annotated,reads_unique,reads_multimapped,max_overhang,present_in_samples,present_in_samples_fraction,has_nested_intron,nested_intron_indices
55715,chr16,23192261,23192982,+,1,0,1841,0,50,16,1.0,False,[]
55717,chr16,23193143,23193498,+,1,0,1064,0,50,16,1.0,False,[]
55720,chr16,23193567,23200985,+,1,0,792,0,50,16,1.0,False,[]
55724,chr16,23201214,23203490,+,1,0,893,0,50,16,1.0,False,[]
